In [ ]:
from google.colab import drive
drive.mount("/content/drive")

basicpath = "/content/drive/MyDrive/졸프"

import pandas as pd
import numpy as np
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# meta_data.json 파일 processing 하기

In [ ]:
import json # json 파일을 읽어오기 위함

In [ ]:
meta_data_file = "meta_data.json"

In [ ]:
with open(os.path.join(basicpath, meta_data_file), "r", encoding="utf-8") as json_file:
    json_string = json_file.readline()
    i = 0
    for _ in range(5):
        json_data = json.loads(json_string)
        print(json_data)
        json_string = json_file.readline()
    # while json_string:
    #   json_data = json.loads(json_string)
    #   if json_data['title'] == "":
    #       i += 1
    #   json_string = json_file.readline()
    # print(i)

{'category': [], 'tech1': '', 'description': ['It is a biology book with God&apos;s perspective.'], 'fit': '', 'title': 'Biology Gods Living Creation Third Edition 10 (A Beka Book Science Series)', 'also_buy': ['0669009075', 'B000K2P5SA', 'B00MD4G2N0', 'B000ASIPTK', '0130508470', '1892427524', '0321567919', 'B000BJBH20', '0547484631', 'B000HAJTQO', 'B000AUCX7I', '0130365645', 'B000BI1Y2O', '0395976715', '052817729X', '1579246443', 'B001CK63XK', '1591669847', '0395879884', '836585161X', 'B01J2F9BH6', 'B00KYEHR4E', '158008141X', '1857928393', '0927545829', 'B015AR0RA0', 'B000TVHHRE', '0865167990', '1579246052', 'B003NXXVD4', 'B000OH6AX0', '061802087X', 'B000NU2X02', '0743252012'], 'tech2': '', 'brand': 'Keith Graham', 'feature': [], 'rank': '1,349,781 in Books (', 'also_view': ['0019777701', 'B000AUCX7I', 'B000K2P5SA', 'B001CK63XK', 'B01J2F9BH6', 'B000BI1Y2O', '1932012540', 'B0095ZCRCK'], 'main_cat': 'Books', 'similar_item': '', 'date': '', 'price': '$39.94', 'asin': '0000092878', 'image

**attribute의 의미 및 종류 / 그리고 삭제할 attribute 선정 및 기준 (preprocessing)**

item의 특성을 파악하여 유저 개개인 간의 선호도를 반영한 content based 방식을 사용할 것이다.

-----

**category : 아이템에 대한 분류 인것 같다. -> 선호도 조사에 적합하다고 판단됨.**

tech1 : 무엇인지도 모르겠지만 대부분이 빈칸이다 -> 삭제

description : 아이템에 대한 묘사 같지만, 문자열로 되어있어 유사점을 찾기가 힘들 것 같고, 이는 category와 비슷한 attribute라고 판단된다. -> 삭제

fit : 이 또한 아무런 정보가 없다. 거의 빈칸이다. -> 삭제

**title : 아이템의 제목이다. 이는 나중에 유저에게 추천할 때, 책의 고유번호로 추천하는 것도 좋지만, 그러면 실질적으로 유저입장에서 어떤 책인지 구분하기 힘들기 때문에, 우선은 데이터로써 가져가는 방향으로 해봐야겠다.**

also_buy : 무엇이랑 같이 샀냐 에 대한 정보인데, 이것이 만약 유저에 대해서 어떤 유저가 이 아이템을 샀을 때 같이 산 물건에 대한 정보가 있었다면, 유의미하게 사용되었을지 모르나, 이는 단순히 이 아마존의 모든 유저 중 이 물건을 산 유저들이 같이 샀던 다른 물건들을 보여준 것이다. 아이템 자체의 특성이라고 볼 수 없다. -> 삭제

tech2' : tech1 과 같은 이유 -> 삭제

**brand : 브랜드 선호도 라는 말이 존재하듯, 브랜드 따라 물건을 구매하는 경우도 있기 때문에, 이는 선호도 조사에 적합하다고 판단된다.**

feature : 의미를 알 수 없고, missing value()로 대부분 이루어져 있다. -> 삭제

rank : 책의 순위는 의미가 없다고 생각한다 -> 삭제

also_view : 이 또한 also_buy attribute와 마찬가지로, 만약 어떤 한 유저가 이 아이템을 샀을 때, 같이 봤던 물건이라면, pre use preference 부분에서 유의미하게 좋은 점수를 줄 수도 있었지만, 이는 지금 collaborating filtering 방식이 아니기 때문에 사용될리 없다. 그리고 아이템 자체의 특성이 아니다. -> 삭제

main_cat : item을 책에 대해서만 다루고 있기 때문에, 필요없다고 생각한다. -> 삭제 

similar_item : 이 또한, 아이템의 특성과는 관련없어보인다. -> 삭제

date : missing value가 매우 많고(), 데이터 설명 사이트에도 설명에 없다. 출판날짜 같은 의미 같은데 결측값이 많은 것으로 보아 의미가 있을 것 같진 않다. -> 삭제

price : missing value가 있긴 하지만, 책을 사는 것은 돈과 큰 관련성이 없어보인다. 예를 들어 옷같은 경우는, 부자들은 명품도 아무렇지도 않게 사겠지만, 가난한 사람들은 그럴 수 없는 경우가 있다. 따라서 선호도 차원에서 고려해야할 대상이 맞겠지만, 책같은 경우는 베스트셀러라고 비싸게 판매되고 그렇지 않다. 그렇기 때문에 가격은 선호도 측면에서 고려대상이 아니라고 생각한다.

**asin : 아이템의 고유번호 이므로 무조건 채용**

imageURL : missing value가 매우 많고() 하지만, 100만개 조사결과 이미지가 있는 item은 300여개 뿐, 이미지는 딱히 선호도 조사에 쓰이지 않음 -> 삭제

imageURLHighRes : imageURL과 마찬가지의 이유 -> 삭제

In [ ]:
output = open(os.path.join(basicpath, 'meta_data_delete_not_need_attribute.json'), 'w', encoding='utf-8')
with open(os.path.join(basicpath, meta_data_file), "r", encoding="utf-8") as json_file:
    i = 0
    json_string = json_file.readline()
    while json_string:
        data = json.loads(json_string)
        preprocess_data = {}
        preprocess_data['asin'] = data['asin']
        preprocess_data['title'] = data['title']
        preprocess_data['brand'] = data['brand']
        if len(data['category']) == 0:
            preprocess_data['category'] = np.nan
        else:
            if data['category'][len(data['category'])-1] == 'Books':
                preprocess_data['category'] = np.nan
            else:
                preprocess_data['category'] = data['category'][len(data['category'])-1]
        preprocess_str = json.dumps(preprocess_data)
        if preprocess_str[-1:] != '}':
            print(print(preprocess_str))
            break
        output.write(preprocess_str + "\n")
        json_string = json_file.readline()
        i += 1
    print(i)
output.close()

2934949


In [ ]:
meta_process_file = 'meta_data_delete_not_need_attribute.json'
with open(os.path.join(basicpath, meta_process_file), "r", encoding="utf-8") as json_file:
    i = 0
    json_string = json_file.readline()
    data_list = []
    while json_string:
    # for _ in range(10):
        data = json.loads(json_string)
        data_list.append([data['asin'],data['title'],data['brand'], data['category']])
        json_string = json_file.readline()
        i += 1
    df = pd.DataFrame(data_list, columns=['item_id', 'title', 'brand', 'category'])
    print(i)

2934949


In [ ]:
df

,item_id,title,brand,category
0,0000092878,Biology Gods Living Creation Third Edition 10 ...,Keith Graham,NaN
1,000047715X,Mksap 16 Audio Companion: Medical Knowledge Se...,Acp,Medicine & Health Sciences
2,0000004545,"Flex! Discography of North American Punk, Hard...",Burkhard Jarisch,Music
3,0000013765,Heavenly Highway Hymns: Shaped-Note Hymnal,Stamps/Baxter,Music
4,0000000116,Georgina Goodman Nelson Womens Size 8.5 Purple...,,NaN
...,...,...,...,...
2934944,B01HJBPTUI,Made Men: A Thriller (Law of Retaliation Book ...,Visit Amazon's Bradley Ernst Page,Thrillers & Suspense
2934945,B01HJC63ZM,Raptor&#39;s Desire (A Planet Desire novelette...,Visit Amazon's Delilah Devlin Page,Erotica
2934946,B01HJEB422,"LG K4 Case,LG Optimus Zone 3 Case,LG Spree Cas...",,NaN
2934947,B01HJDS76Y,Magickal Incantations,SilvrMyst,New Age & Spirituality


In [ ]:
df_replace_nan = df.replace("", np.nan)

In [ ]:
df_replace_nan.isnull().sum()

item_id          0
title          827
brand       100982
category    389429
dtype: int64

In [ ]:
df_replace_nan

,item_id,title,brand,category
0,0000092878,Biology Gods Living Creation Third Edition 10 ...,Keith Graham,NaN
1,000047715X,Mksap 16 Audio Companion: Medical Knowledge Se...,Acp,Medicine & Health Sciences
2,0000004545,"Flex! Discography of North American Punk, Hard...",Burkhard Jarisch,Music
3,0000013765,Heavenly Highway Hymns: Shaped-Note Hymnal,Stamps/Baxter,Music
4,0000000116,Georgina Goodman Nelson Womens Size 8.5 Purple...,NaN,NaN
...,...,...,...,...
2934944,B01HJBPTUI,Made Men: A Thriller (Law of Retaliation Book ...,Visit Amazon's Bradley Ernst Page,Thrillers & Suspense
2934945,B01HJC63ZM,Raptor&#39;s Desire (A Planet Desire novelette...,Visit Amazon's Delilah Devlin Page,Erotica
2934946,B01HJEB422,"LG K4 Case,LG Optimus Zone 3 Case,LG Spree Cas...",NaN,NaN
2934947,B01HJDS76Y,Magickal Incantations,SilvrMyst,New Age & Spirituality


In [ ]:
i = 0
for idx in df.index:
    i += 1
    if len(df.loc[idx,'item_id']) != 10: 
        print(df.loc[idx,'item_id'])
print(i)

2934949


상관계수 구하려면 nan 값도 없어야하고 category에 [] 형태로 들어간 값도 계산못해야할텐데 그럼 삭제하는 방법밖에 없을까?? - 고민중

In [ ]:
df_replace_nan.corr()

""


category 가 list로 되니, [] 즉, missing value 인 곳에 np.nan이 들어갈 수 없고, 라벨인코딩도 안되어서 그냥 가장 안쪽 category로 변경해줌.

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
for col in df.columns:
    df_replace_nan[col] = labelencoder.fit_transform(df_replace_nan[col])

df_replace_nan.head()

,item_id,title,brand,category
0,8,372413,443909,1251
1,18,1471132,9529,728
2,3,856765,95441,777
3,4,1027403,744955,777
4,0,928918,1421221,1251


In [ ]:
df_replace_nan.head()

In [ ]:
df_replace_nan

,item_id,title,brand,category
0,8,372413,443909,1251
1,18,1471132,9529,728
2,3,856765,95441,777
3,4,1027403,744955,777
4,0,928918,1421221,1251
...,...,...,...,...
2934944,2930018,1378395,870518,1148
2934945,2930020,1784346,940436,425
2934946,2930022,1263221,1421221,1251
2934947,2930021,23924,738059,814


nan 값도 같은 숫자로 바뀜 -> corr에 nan끼리 상관관계 높게 나올 확률이 있음 -> 그럼 nan 값을 삭제해줘야하는가?

아니면 상관계수는 구하지 않아도 될까?